In [1]:
#Importing necessary libraries

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from pandas import DataFrame , concat
from sklearn.metrics import mean_absolute_error , mean_squared_error
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,LSTM,Activation,Dropout
from tensorflow.keras.optimizers import Adam
from numpy import array , hstack
from tensorflow import keras
import tensorflow as tf
import datetime as dt
from datetime import datetime
from keras.models import Sequential
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.optimizers import Adam
import sklearn.metrics as metrics

%matplotlib inline
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

# Supressing warnings
import warnings
warnings.filterwarnings('ignore')

In [13]:
with open("methanol_80%train.json","r") as json_file:
    json_savedModel =json_file.read()
    
model =tf.keras.models.model_from_json(json_savedModel)
model.load_weights("weights_80%train.h5")

# Compiling the Neural Network
model.compile(optimizer = Adam(learning_rate=0.01), loss='mean_squared_error', metrics=['mse','mae','mape'])

In [14]:
# Defining mean absolute percentage error
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [15]:
df = pd.read_csv("master methanol.csv")
#df = df.drop('North_America',axis=1)
df.tail()

,Date,India Price,Asia Price,China Price,Rotterdam Price,US Domestic Price,Natural Gas Price,Crude Oil Price
687,11-06-2021,294.0,353.0,309.8,305.0,121.13,3.12,71.5
688,18-06-2021,294.0,355.0,308.6,310.8,121.13,3.28,71.5
689,25-06-2021,292.0,364.0,307.0,309.5,121.13,3.28,71.5
690,02-07-2021,292.0,368.0,308.0,313.2,121.13,3.11,74.8
691,09-07-2021,290.0,381.0,306.4,334.7,121.13,3.11,74.8


In [16]:
crude_oil = df[['Date',"Crude Oil Price"]]
crude_oil.head()
natural_gas = df[['Date',"Natural Gas Price"]]
natural_gas.head()

,Date,Natural Gas Price
0,11-04-2008,9.88
1,18-04-2008,10.13
2,25-04-2008,10.54
3,02-05-2008,10.75
4,09-05-2008,11.11


In [17]:
test_data = df.iloc[500:,:]
test_data
# testing
l = [10,15,20,30,40,50,60,70,80,90,100,110,120,130,140,150,160,170,180,190,200,210,220,210,220,230,240,250,260,270,280,290,300,400]
len(l)

34

In [18]:
list_error = []
for i  in l :
    master = df.iloc[:-i,:]
    master.tail()
    crude_p= crude_oil.iloc[-i-1:,:]      # for future forecasting
    crude_p = crude_p.head(6)
    natural_p= natural_gas.iloc[-i-1:,:]      # for future forecasting
    natural_p = natural_p.head(6)
    test= df.iloc[-i-1:,:]         # for testing
    test = test.head(6)

    natural_p = natural_p['Natural Gas Price'].values
    natural_p = np.expand_dims(natural_p,axis=1)

    crude_p  =  crude_p['Crude Oil Price'].values
    crude_p = np.expand_dims(crude_p,axis=1)

# Select features (columns) to be involved into training and predictions
    from sklearn.preprocessing import MinMaxScaler
    dataset_train = master
    cols = list(dataset_train)[1:8]

# Extract dates (will be used in visualization)
    datelist_train = list(dataset_train['Date'])

#Displaying the values
    #print('Training set shape == {}'.format(dataset_train.shape))
    #print('All timestamps == {}'.format(len(datelist_train)))
    #print('Featured selected: {}'.format(cols))


#Preparing the training set
    dataset_train = dataset_train[cols].astype(str)
    for i in cols:
        for j in range(0, len(dataset_train)):
            dataset_train[i][j] = dataset_train[i][j].replace(',', '')

    dataset_train = dataset_train.astype(float)
    training_set = dataset_train.to_numpy()

    #print('Shape of training set == {}.'.format(training_set.shape))



    natural_train =master['Natural Gas Price'].values
    natural_train  = np.expand_dims(natural_train,axis=1)
    crude_train = master['Crude Oil Price'].values
    crude_train = np.expand_dims(crude_train,axis=1)


# Feature scaling using standard scaler
    from sklearn.preprocessing import StandardScaler

    sc = StandardScaler()
    sc.fit(training_set)                                  #fitting on training data
    training_set_scaled=sc.transform(training_set)


    sc_nat = StandardScaler()                   #fitting on forecasting data
    sc_crude =StandardScaler()
    sc_nat.fit(natural_train)               # fitting on training natural gas
    sc_crude.fit(crude_train)               # fitting on training crude oil
    crude = sc_crude.transform(crude_p)     # transforming future 12 weeks data crude oil
    natural =sc_nat.transform(natural_p)    # transforming future 12 weeks data natural gas

    sc_predict = StandardScaler()
    sc_predict.fit(training_set[:,:5])
    sc_predict.transform(training_set[:,:5])
#crude_future = sc_crude.transform(y)
#natural_future = sc_natural.fit_transform(y1)

# Creating a data structures for training

    X_train = []
    y_train = []

#n_future = 20   # Number of weeks we want top predict into the future
    n_past =20   # Number of weeks we want to use to predict the future

    for i in range(n_past, len(training_set_scaled)):
        X_train.append(training_set_scaled[i - n_past:i, 0:dataset_train.shape[1] ])  #dataset_train.shape[1]
        y_train.append(training_set_scaled[i , :5]) 

    X_train, y_train = np.array(X_train), np.array(y_train)

    #print('X_train shape == {}.'.format(X_train.shape))
    #print('y_train shape == {}.'.format(y_train.shape))


    x= X_train[-1:]
    predict = model.predict(x)
#sc_predict.inverse_transform(predict)
    #print(predict.shape)
    #print(sc_predict.inverse_transform(predict))
    forecast = []
    forecast.append(predict)
    for i in range(5):
        result = np.delete(x, 0, 1)
        result  =np.squeeze(result,axis=0)
        p = np.squeeze(predict,axis=0)
        p1 =np.concatenate((p,natural[i],crude[i]))
        p1 =np.expand_dims(p1,axis=0)
        new_predict = np.concatenate((result,p1))
        new_predict = np.expand_dims(new_predict,axis=0)
        predict = model.predict(new_predict)
        forecast.append(predict)
        x = new_predict
    
    


    fore = np.array(forecast)
    forecast=sc_predict.inverse_transform(fore)


    forecast = np.squeeze(forecast)
    predictions_future = pd.DataFrame(forecast, columns=['India Pred Price', 'Asia Pred Price',"China Pred Price","Rotterdam Pred Price",'US Domestic Pred Price',
                                 ])


    test.reset_index(drop=True,inplace=True)
    output= pd.concat([test,predictions_future],axis=1)
#output.head()
    mae = metrics.mean_absolute_error(np.array(output['India Price']),np.array(output['India Pred Price']))
    mse = metrics.mean_squared_error(np.array(output['India Price']),np.array(output['India Pred Price']))
    #mape = mean_absolute_percentage_error(np.array(output.iloc['India Price']),np.array(output.iloc['India Pred Price']))
    print("Mean absolute error of Forecasted data set (India Price):     {}".format(mae))
    #print("Mean absolute % error of Forecasted dataset :                 {}".format(mape))
    #print("Mean squared error of error of Forecasted dataset :           {}".format(mse))
    #print("*"*50)
    list_error.append(round(mae,2))
    #output = output.rename(columns={"India Price":"Actual","India Pred Price":"Predicted"})
    #output = output.rename(columns={"India Price":"Actual","India Pred Price":"Predicted"})
    output['Error'] =output['India Price']-output['India Pred Price']
    output = output.rename(columns={"India Price":"Actual","India Pred Price":"Predicted"})
    print(output[['Date','Actual','Predicted','Error']])
    print("*"*50)





Mean absolute error of Forecasted data set (India Price):     27.5704345703125
         Date  Actual   Predicted      Error
0  30-04-2021   276.0  269.017242   6.982758
1  07-05-2021   276.0  266.880798   9.119202
2  14-05-2021   296.0  263.211304  32.788696
3  21-05-2021   295.0  259.367523  35.632477
4  28-05-2021   296.0  255.655273  40.344727
5  04-06-2021   293.0  252.445251  40.554749
**************************************************
Mean absolute error of Forecasted data set (India Price):     17.341529846191406
         Date  Actual   Predicted      Error
0  26-03-2021   274.0  267.579163   6.420837
1  02-04-2021   274.0  264.227661   9.772339
2  09-04-2021   276.0  259.649109  16.350891
3  16-04-2021   275.0  255.047653  19.952347
4  23-04-2021   275.0  251.263245  23.736755
5  30-04-2021   276.0  248.183990  27.816010
**************************************************
Mean absolute error of Forecasted data set (India Price):     30.038551330566406
         Date  Actual   Pre

Mean absolute error of Forecasted data set (India Price):     105.98286946614583
         Date  Actual   Predicted       Error
0  17-11-2017   325.0  296.682434   28.317566
1  24-11-2017   354.0  290.447571   63.552429
2  01-12-2017   381.0  282.484467   98.515533
3  08-12-2017   401.0  274.440125  126.559875
4  15-12-2017   415.0  266.986603  148.013397
5  22-12-2017   432.0  261.061584  170.938416
**************************************************
Mean absolute error of Forecasted data set (India Price):     68.7935078938802
         Date  Actual   Predicted      Error
0  08-09-2017   310.0  262.463715  47.536285
1  15-09-2017   350.0  262.862793  87.137207
2  22-09-2017   329.0  260.831055  68.168945
3  29-09-2017   330.0  258.218536  71.781464
4  06-10-2017   331.0  255.849228  75.150772
5  13-10-2017   316.0  253.013626  62.986374
**************************************************
Mean absolute error of Forecasted data set (India Price):     15.779513041178385
         Date  Actua